In [3]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', '기사 제목 본문 연관성 판별']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        part = source_data.name.split('_')[1][-1]
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            title = line['sourceDataInfo']['newsTitle'] + ' ' + line['sourceDataInfo']['newsSubTitle']
            content = line['sourceDataInfo']['newsContent']
            sentences = line['sourceDataInfo']['sentenceInfo']
            sentences = [sentence['sentenceContent'] for sentence in sentences]
            #### data preprocess end 

            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            text = title + ' ' + content
            data['text'] = text
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### LM(한국어) end

            
            #### 기사 제목 본문 연관성 판별
            data = {'input': {}, 'output': None}
            ## preprocess data from line
            if part == '1':
                title = line['labeledDataInfo']['newTitle']
            data['input']['title'] = title
            data['input']['sentences'] = sentences
            if part == '1':
                data['output'] = line['labeledDataInfo']['referSentenceInfo']
                data['output'] = [s['referSentenceyn'] for s in data['output']]
            elif part == '2':
                data['output'] = line['labeledDataInfo']['processSentenceInfo']
                data['output'] = [s['subjectConsistencyYn'] for s in data['output']]
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 기사 제목 본문 연관성 판별 end


            
            # break
        # break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 42/42 [00:05<00:00,  7.82it/s]
